In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import tabulate
from collections import  namedtuple
from statsmodels.stats.power import TTestIndPower, TTestPower, zt_ind_solve_power
# from autoviz.AutoViz_Class import AutoViz_Class
# %matplotlib inline

### Odkazy na relevantní materiály
- [Statistické testy ve scipy](https://docs.scipy.org/doc/scipy/reference/stats.html#statistical-tests)
- [Statsmodels dokumentace](https://www.statsmodels.org/dev/index.html)
- [Skripta](https://dostal.vyzkum-psychologie.cz/skripta_statistika.pdf)
- [Zadání příkladů](https://dostal.vyzkum-psychologie.cz/stat/index.php)

# Úkol 13: Testy kontingenčních tabulek
Provádíme internetovou anketu mezi studenty Univerzity Palackého a tážeme se jich na několik otázek souvisejících s možností zavedením školného pro vysokoškolské studenty. Podařilo se nám získat soubor 220 odpovědí. V datové tabulce níže je odpověď na otázku, zdali by za předpokladu zavedení prospěchových stipendií, zkvalitnění výuky a možnosti výhodných studentských půjček, respondent schvaloval zavedení školného (1 je souhlas, 0 nesouhlas).

Otázkou je, jestli je náš nenáhodný výběr reprezentativní. Mohli bychom například pochybovat, jestli jsou mezi respondenty zastoupeny jednotlivé fakulty UPOL v poměru, který odpovídá jejich počtům studentů. Z výroční zprávy UPOL jsme zjistili, že fakulty mají, co se týče počtů studentů, tyto proporce:

CMTF: 5 %
LF: 10 %
FF: 26 %
PřF: 19 %
PedF: 20 %
FTK: 9 %
PF: 8 %
FZV: 3 %

Otestujte níže uvedené hypotézy a odpovězte na otázky. Není-li řečeno jinak, hypotézy testujte pomocí statistiky "chí kvadrát" (Z ve skriptech) a uvádějte jako své odpovědi její hodnotu.

---

Načtení a vizualizace dat

In [2]:
fakulta = ["CMTF", "LF", "FF", "PrF", "PedF", "FTK", "PF", "FZV"]
true_ratio = [0.05, 0.1, 0.26, 0.19, 0.2, 0.09, 0.08, 0.03]

assert sum(true_ratio) == 1, sum(true_ratio)

df_true = pd.DataFrame(index=fakulta, data={"ratio_expected": true_ratio})
df_true

,ratio_expected
CMTF,0.05
LF,0.10
FF,0.26
PrF,0.19
PedF,0.20
FTK,0.09
PF,0.08
FZV,0.03


In [3]:
df = pd.read_csv("data.csv", index_col=0, delimiter=";")
df

,Fakulta,Pohlavi,Souhlas
#,,,
1,LF,1,1
7,FF,0,0
11,FTK,1,1
15,PedF,0,0
16,PrF,0,0
...,...,...,...
967,PrF,0,0
968,CMTF,0,0
986,CMTF,0,0


In [4]:
df.describe()

,Pohlavi,Souhlas
count,220.000000,220.000000
mean,0.227273,0.281818
std,0.420026,0.450911
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,1.000000
max,1.000000,1.000000


In [5]:
df["Fakulta"].unique()

array(['LF', 'FF', 'FTK', 'PedF', 'PrF', 'CMTF', nan, 'FZV', 'PF'],
      dtype=object)

---

Pravděpodobnost toho, že respondent navštěvuje danou fakultu, odpovídá relativní četnosti studentů této fakulty v rámci UP. (Tzn. zastoupení studentů jednotlivých fakult v souboru je reprezentativní. Z analýzy vyřaďte studenty, co fakultu neuvedli. Nepoužívejte korekci na spojitost.)

In [6]:
df_filter = df[~df["Fakulta"].isna()]

df_counts = df_filter[["Fakulta", "Souhlas"]].groupby("Fakulta").agg("count").rename(columns={"Souhlas": "count_observed"})
df_counts["ratio_observed"] = df_counts["count_observed"] / len(df_filter)
df_counts = df_counts.join(df_true)
df_counts["count_expected"] =  df_counts["ratio_expected"] * len(df_filter)
df_counts = df_counts.sort_values("count_expected", ascending=False)
df_counts


,count_observed,ratio_observed,ratio_expected,count_expected
Fakulta,,,,
FF,71,0.349754,0.26,52.78
PedF,27,0.133005,0.20,40.60
PrF,54,0.266010,0.19,38.57
LF,20,0.098522,0.10,20.30
FTK,8,0.039409,0.09,18.27
PF,14,0.068966,0.08,16.24
CMTF,6,0.029557,0.05,10.15
FZV,3,0.014778,0.03,6.09


In [7]:
def chi2(counts, counts_expected, dof, correction="no_correction"):
        
    if correction == "no_correction":
        normalized_errors = (counts - counts_expected)**2 / counts_expected
    elif correction == "yates":
        normalized_errors = (np.abs(counts - counts_expected) - 0.5)**2 / counts_expected
    else:
        raise ValueError(f"{correction=} unknown")
    
#     display(counts)
#     display(counts_expected)
#     display(normalized_errors)
    
    stat = sum(normalized_errors)
    p = 1-stats.chi2.cdf(stat, df=dof)
    return stat, p
    
dof = len(df_counts) - 1

stat, p = chi2(df_counts["count_observed"], df_counts["count_expected"], dof)
print(f"{stat=}, {p=}")



stat=26.369167835592517, p=0.00043260596526772943


[(Skripta, str. 134)](https://dostal.vyzkum-psychologie.cz/skripta_statistika.pdf#page=134)

Relativní četnosti studentů, co souhlasí se zavedením školného, se napříč fakultami liší. (Fakulty, u nichž vychází očekávaná četnost nižší než 5, slučte do kategorie ostatní. Pokud by očekávaná četnost byla u této skupiny stále nižší než 5, nezahrnujte ji do analýzy. Studenty, co fakultu neuvedli, vyřaďte. Nepoužívejte korekci na spojitost.)

In [8]:
# Incorrect results for the commented code
# vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv

# df_filter = df[~df["Fakulta"].isna()]
# faculties_to_merge = ["PF", "FTK", "CMTF", "FZV"]
# df_filter.loc[df_filter["Fakulta"].isin(faculties_to_merge), "Fakulta"] = "Other"

# df_counts = df_filter[["Fakulta", "Souhlas"]] \
#     .groupby("Fakulta") \
#     .agg(["sum", "count", "mean"])["Souhlas"] \
#     .rename(columns={
#         "mean": "ratio_observed", 
#         "sum": "count_observed",
#         "count": "n",
#     })
# df_counts["ratio_expected"] = sum(df_filter["Souhlas"]) /  len(df_filter)
# df_counts["count_expected"] = df_counts["n"] * df_counts["ratio_expected"]
# df_counts = df_counts.sort_values("count_expected", ascending=False)

# display(df_counts)


In [9]:
# dof = len(df_counts) - 1

# p = stats.chisquare(df_counts["count_observed"], df_counts["count_expected"], ddof=0)
# print(p)

# stat, p = chi2(df_counts["count_observed"], df_counts["count_expected"], dof)
# print(f"{stat=}, {p=}")
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [10]:
def get_observed_and_expected_frequencies(df, index, columns):
    df_counts_observed = df.pivot_table(index=index, columns=columns, values=None,  aggfunc="count")

    # Prepare marginal frequencies for expected counts
    index_marginal = np.sum(df_counts_observed.values, axis=1, keepdims=True)
    columns_marginal = np.sum(df_counts_observed.values, axis=0, keepdims=True)
    n = len(df)

    # Calculate expected counts
    counts_expected = np.matmul(index_marginal, columns_marginal) / n
    
    df_counts_expected = pd.DataFrame(data=counts_expected, index=df_counts_observed.index, columns=df_counts_observed.columns)

    return df_counts_observed, df_counts_expected

In [11]:
df_filter = df[~df["Fakulta"].isna()]

# Remove Faculties whose expected counts < 5 (see the table)
faculties_to_merge = ["PF", "FTK", "CMTF", "FZV"]
df_filter.loc[df_filter["Fakulta"].isin(faculties_to_merge), "Fakulta"] = "Other"

df_counts_observed, df_counts_expected = get_observed_and_expected_frequencies(df_filter, index="Fakulta", columns="Souhlas")
dof = np.prod(np.array(df_counts_observed.shape) - 1)  # (r-1)(s-1)

print("Pozorovaná četnost")
display(df_counts_observed)

print("Očekávaná četnost")
display(df_counts_expected)


stat, p = chi2(df_counts_observed.values.ravel(), df_counts_expected.values.ravel(), dof=dof)
print(f"{stat=}, {p=}")
    

Pozorovaná četnost


Pohlavi    
Souhlas       0   1
Fakulta            
FF           50  21
LF           16   4
Other        18  13
PedF         24   3
PrF          40  14

Očekávaná četnost


Pohlavi           
Souhlas          0          1
Fakulta                      
FF       51.763547  19.236453
LF       14.581281   5.418719
Other    22.600985   8.399015
PedF     19.684729   7.315271
PrF      39.369458  14.630542

stat=7.71714786613469, p=0.1025065102666789


Vyčíslete sílu závislosti proměnných fakulta a souhlas pomocí koeficientu fí.

In [12]:
def get_phi_coefficient(z_stat, n):
    return np.sqrt(z_stat / n)

phi = get_phi_coefficient(stat, n=len(df_filter))
print(f"{phi=}")

phi=0.1949756567618828


Zastoupení studentů, co souhlasí se zavedením školného, je rozdílné v podsouboru mužů a žen. (Uveďte p-hodnotu. Pracujte s celým souborem. Nepoužívejte korekci na kontinuitu.)

In [13]:
# Using full dataset
df_filter = df
# df_filter = df[~df["Fakulta"].isna()]
df_filter.loc[df_filter["Fakulta"].isna(), "Fakulta"] = "Unknown"

df_counts_observed, df_counts_expected = get_observed_and_expected_frequencies(df_filter, index="Pohlavi", columns="Souhlas")
dof = np.prod(np.array(df_counts_observed.shape) - 1)  # (r-1)(s-1)

print("Pozorovaná četnost")
display(df_counts_observed)

print("Očekávaná četnost")
display(df_counts_expected)


stat, p = chi2(df_counts_observed.values.ravel(), df_counts_expected.values.ravel(), dof=dof)
print(f"{stat=}, {p=}, {dof=}")

Pozorovaná četnost


Fakulta    
Souhlas       0   1
Pohlavi            
0           121  49
1            37  13

Očekávaná četnost


Fakulta           
Souhlas           0          1
Pohlavi                       
0        122.090909  47.909091
1         35.909091  14.090909

stat=0.15218696706939253, p=0.6964541140067726, dof=1


Zastoupení studentů, co souhlasí se zavedením školného, je rozdílné v podsouboru mužů a žen. (Uveďte p-hodnotu. Pracujte s celým souborem. Použijte Yatesovu korekci na kontinuitu.)

In [14]:
stat, p = chi2(df_counts_observed.values.ravel(), df_counts_expected.values.ravel(), dof=dof, correction="yates")
print(f"{stat=}, {p=}, {dof=}")

stat=0.04465207887973507, p=0.8326450796969181, dof=1


Zastoupení studentů, co souhlasí se zavedením školného, je rozdílné v podsouboru mužů a žen. (Uveďte oboustrannou p-hodnotu získanou Pomocí Fisherova faktoriálového testu.)